Install needed libraries

In [2]:
%pip install requests
%pip install PyGithub
%pip install openai
%pip install --upgrade openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Imports

In [1]:
from github import Github
from openai import OpenAI, AzureOpenAI
import json
import requests
import re
from dotenv import load_dotenv
import os

Function to check if input is already a GitHub repo

In [2]:
def is_github_repo_link(env):
    return env.startswith("https://github.com/") or env.startswith("http://github.com/")

Prompt for user

In [3]:
prompt = input("Enter a name of the environment or link to GitHub repo: ")
flag = True
if is_github_repo_link(prompt) == False:
   flag = False

Function for searching GitHub repositories

In [4]:
def retrieve_files(repo_urls):
    file_names = ['README.md', 'install.md', 'package.json']
    files_content = {file_name: [] for file_name in file_names}

    for url in repo_urls:
        if url.endswith('.git'):
            repo_name = url[:-4].split('github.com/')[-1]
        else:
            repo_name = url.split('github.com/')[-1]

        branches = ['main', 'master']
        for branch in branches:
            api_url = f"https://api.github.com/repos/{repo_name}/git/trees/{branch}?recursive=1"

            try:
                response = requests.get(api_url)
                if response.status_code == 200:
                    tree = response.json().get('tree', [])
                    for file_info in tree:
                        file_path = file_info['path']
                        file_name = file_path.split('/')[-1]
                        if file_name in file_names:
                            file_url = f"https://raw.githubusercontent.com/{repo_name}/{branch}/{file_path}"
                            file_response = requests.get(file_url)
                            if file_response.status_code == 200:
                                files_content[file_name].append(file_response.text)
                                print(f"Successfully retrieved {file_name} from {file_url}")
                            else:
                                print(f"Failed to retrieve {file_name} from {file_url}. Status code: {file_response.status_code}")
                    break  # Exit loop if successful
                elif response.status_code == 404:
                    print(f"Branch '{branch}' not found for repository {repo_name}.")
                else:
                    print(f"Failed to retrieve file list from {api_url}. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error while trying to get file list from {api_url}: {e}")

    return files_content

OpenAI API for getting repositories

In [6]:
load_dotenv()
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)
def generate_github_repos(prompt, model="gpt-4o"):
    system_prompt = "You are an AI assistant that suggests relevant GitHub repositories based on a project description."
    user_prompt = f"Suggest GitHub repositories for a project that involves the following technologies: {prompt}. Provide only the URLs of the repositories."

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    suggested_repos = response.choices[0].message.content.strip()
    repo_urls = re.findall(r'(https?://github\.com/[^\s]+)', suggested_repos)
    return repo_urls
repo_urls = generate_github_repos(prompt)
if repo_urls:
    print(repo_urls)
else:
    print("Failed to fetch repository URLs.")

['https://github.com/angular/angular', 'https://github.com/ngrx/platform', 'https://github.com/angular/angular-cli', 'https://github.com/angular/universal', 'https://github.com/preboot/angular-webpack']


Analyze and pre-process the data

In [20]:
files_content = retrieve_files(repo_urls)

Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.circleci/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.devcontainer/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/angular/angular/main/.github/actions/deploy-docs-site/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.vscode/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.yarn/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/scripts/update-cli-help/README.md
Successfully retrieved README.md from https://raw.githubuserconte

In [27]:
def parse_readme(content):
    sections = {}
    current_section = None
    lines = content.split('\n')

    for line in lines:
        header_match = re.match(r'^(#+)\s+(.*)', line)
        if header_match:
            level = len(header_match.group(1))
            section_name = header_match.group(2).strip()
            current_section = section_name
            sections[current_section] = []
        elif current_section:
            sections[current_section].append(line)

    for section in sections:
        sections[section] = "\n".join(sections[section])

    return sections

In [32]:
def parse_package_json(content):
    try:
        if not content.strip():
            print("Empty content for package.json.")
            return {}

        package_data = json.loads(content)
        return {
            "dependencies": package_data.get("dependencies", {}),
            "devDependencies": package_data.get("devDependencies", {}),
            "scripts": package_data.get("scripts", {})
        }
    except json.JSONDecodeError as e:
        print(f"Error parsing package.json: {e}")
        return {}

In [33]:
def collect_context(files_content):
    context = {}

    for file_name, contents in files_content.items():
        if file_name == 'README.md':
            for content in contents:
                context[file_name] = parse_readme(content)
        elif file_name == 'package.json':
            for content in contents:
                context[file_name] = parse_package_json(content)

    return context
collect_context(files_content)

Error parsing package.json: Expecting value: line 1 column 1 (char 0)


{'README.md': {'angular-webpack': "\n[![Dependency Status](https://david-dm.org/preboot/angular-webpack/status.svg)](https://david-dm.org/preboot/angular-webpack#info=dependencies) [![devDependency Status](https://david-dm.org/preboot/angular-webpack/dev-status.svg)](https://david-dm.org/preboot/angular-webpack#info=devDependencies)\n[![Join the chat at https://gitter.im/preboot/angular-webpack](https://badges.gitter.im/preboot/angular-webpack.svg)](https://gitter.im/preboot/angular-webpack?utm_source=badge&utm_medium=badge&utm_campaign=pr-badge&utm_content=badge)\n\nA complete, yet simple, starter for Angular v2+ using Webpack.\n\nThis seed repo serves as an Angular starter for anyone looking to get up and running with Angular and TypeScript fast. Using [Webpack](http://webpack.github.io/) for building our files and assisting with boilerplate. We're also using Protractor for our end-to-end story and Karma for our unit tests.\n* Best practices in file and application organization for [

Generate JSON file for Dev Container with OpenAI API

In [39]:
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

def generate_dev_container_json(context, model="gpt-4o"):
    system_prompt = "You are an AI assistant that creates JSON file for dev container.\n\n"
    user_prompt = f"Generate JSON configuration for dev container based on the context provided: {context}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        print("Raw API Response:")
        print(response)

        if response.choices:
            for choice in response.choices:
                if choice.finish_reason == 'stop' and choice.message:
                    generated_json = choice.message.content.strip()
                    try:
                        # Extracting JSON configuration from the message content
                        start_index = generated_json.find('{')
                        end_index = generated_json.rfind('}') + 1
                        json_str = generated_json[start_index:end_index]
                        parsed_json = json.loads(json_str)

                        print("Generated Dev Container JSON Configuration:")
                        print(json.dumps(parsed_json, indent=4))
                        return parsed_json
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        return None
                else:
                    print(f"Invalid choice received: {choice}")
                    return None
        else:
            print("No valid choices found in API response.")
            return None

    except Exception as ex:
        print(f"Error generating devcontainer.json: {ex}")
        return None
    
context = {
    'README.md': {
        'angular-webpack': "\n[![Dependency Status](https://david-dm.org/preboot/angular-webpack/status.svg)](https://david-dm.org/preboot/angular-webpack#info=dependencies) [![devDependency Status](https://david-dm.org/preboot/angular-webpack/dev-status.svg)](https://david-dm.org/preboot/angular-webpack#info=devDependencies)\n[![Join the chat at https://gitter.im/preboot/angular-webpack](https://badges.gitter.im/preboot/angular-webpack.svg)](https://gitter.im/preboot/angular-webpack?utm_source=badge&utm_medium=badge&utm_campaign=pr-badge&utm_content=badge)\n\nA complete, yet simple, starter for Angular v2+ using Webpack.\n\nThis seed repo serves as an Angular starter for anyone looking to get up and running with Angular and TypeScript fast. Using [Webpack](http://webpack.github.io/) for building our files and assisting with boilerplate. We're also using Protractor for our end-to-end story and Karma for our unit tests.\n* Best practices in file and application organization for [Angular](https://angular.io/).\n* Ready to go build system using [Webpack](https://webpack.github.io/docs/) for working with [TypeScript](http://www.typescriptlang.org/).\n* Testing Angular code with [Jasmine](http://jasmine.github.io/) and [Karma](http://karma-runner.github.io/).\n* Coverage with [Istanbul](https://github.com/gotwarlost/istanbul)\n* End-to-end Angular code using [Protractor](https://angular.github.io/protractor/).\n* Stylesheets with [SASS](http://sass-lang.com/) (not required, it supports regular css too).\n* Error reported with [TSLint](http://palantir.github.io/tslint/) and [Codelyzer](https://github.com/mgechev/codelyzer).\n* Documentation with [TypeDoc](http://typedoc.org/).\n\n>Warning: Make sure you're using the latest version of Node.js and NPM\n",
        'Quick start': '\n```bash',
        'clone our repo': '$ git clone https://github.com/preboot/angular-webpack.git my-app\n',
        'change directory to your app': '$ cd my-app\n',
        'install the dependencies with npm': '$ npm install\n',
        'start the server': '$ npm start\n```\ngo to [http://localhost:8080](http://localhost:8080) in your browser.\n',
        'Table of Contents': '\n* [Getting Started](#getting-started)\n    * [Dependencies](#dependencies)\n    * [Installing](#installing)\n    * [Developing](#developing)\n    * [Testing](#testing)\n    * [Production](#production)\n    * [Documentation](#documentation)\n* [Frequently asked questions](#faq)\n* [TypeScript](#typescript)\n* [License](#license)\n',
        'Getting Started': '',
        'Dependencies': "\nWhat you need to run this app:\n* `node` and `npm` (Use [NVM](https://github.com/creationix/nvm))\n* Ensure you're running Node (`v6.x.x`+) and NPM (`3.x.x`+)\n",
        'Installing': '\n* `fork` this repo\n* `clone` your fork\n* `npm install` to install all dependencies\n',
        'Developing': '\nAfter you have installed all dependencies you can now start developing with:\n\n* `npm start`\n\nIt will start a local server using `webpack-dev-server` which will watch, build (in-memory), and reload for you. The application can be checked at `http://localhost:8080`.\n\nAs an alternative, you can work using Hot Module Replacement (HMR):\n\n* `npm run start:hmr`\n\nAnd you are all set! You can now modify your components on the fly without having to reload the entire page.\n',
        'Testing': '',
        '1. Unit Tests': '\n* single run: `npm test`\n* live mode (TDD style): `npm run test-watch`\n',
        '2. End-to-End Tests (aka. e2e, integration)': '\n* single run:\n  * in a tab, *if not already running!*: `npm start`\n  * in a new tab: `npm run webdriver-start`\n  * in another new tab: `npm run e2e`\n* interactive mode:\n  * instead of the last command above, you can run: `npm run e2e-live`\n  * when debugging or first writing test suites, you may find it helpful to try out Protractor commands without starting up the entire test suite. You can do this with the element explorer.\n  * you can learn more about [Protractor Interactive Mode here](https://github.com/angular/protractor/blob/master/docs/debugging.md#testing-out-protractor-interactively)\n',
        'Production': '\nTo build your application, run:\n\n* `npm run build`\n\nYou can now go to `/dist` and deploy that to your server!\n',
        'Documentation': '\nYou can generate api docs (using [TypeDoc](http://typedoc.org/)) for your code with the following:\n\n* `npm run docs`\n',
        'FAQ': '',
        'Do I need to add script / link tags into index.html ?': "\nNo, Webpack will add all the needed Javascript bundles as script tags and all the CSS files as link tags. The advantage is that you don't need to modify the index.html every time you build your solution to update the hashes.\n",
        'How to include external angular libraries ?': "\nIt's simple, just install the lib via npm and import it in your code when you need it. Don't forget that you need to configure some external libs in the [bootstrap](https://github.com/preboot/angular-webpack/blob/master/src/main.ts) of your application.\n",
        'How to include external css files such as bootstrap.css ?': "\nJust install the lib and import the css files in [vendor.ts](https://github.com/preboot/angular-webpack/blob/master/src/vendor.ts). For example this is how to do it with bootstrap:\n\n```sh\nnpm install bootstrap@next --save\n```\n\nAnd in [vendor.ts](https://github.com/preboot/angular-webpack/blob/master/src/vendor.ts) add the following:\n\n```ts\nimport 'bootstrap/dist/css/bootstrap.css';\n```\n",
        'TypeScript': '\n> To take full advantage of TypeScript with autocomplete you would have to use an editor with the correct TypeScript plugins.\n',
        'Use a TypeScript-aware editor': '\nWe have good experience using these editors:\n\n* [Visual Studio Code](https://code.visualstudio.com/)\n* [Webstorm 11+](https://www.jetbrains.com/webstorm/download/)\n* [Atom](https://atom.io/) with [TypeScript plugin](https://atom.io/packages/atom-typescript)\n* [Sublime Text](http://www.sublimetext.com/3) with [Typescript-Sublime-Plugin](https://github.com/Microsoft/Typescript-Sublime-plugin#installation)\n',
        'License': '\n[MIT](/LICENSE)\n'},
    'package.json': {
        'dependencies': {
            '@angular/common': '~4.2.6',
            '@angular/compiler': '~4.2.6'
        },
        'devDependencies': {
            'typescript': '~2.3.2',
            'webpack': '^2.6.1',
        },
        'scripts': {
            'build': 'webpack',
            'start': 'npm run server',
            'start:hmr': 'npm run server -- --hot',
            'postinstall': 'npm run webdriver-update'
        }
    }
}

# Generiranje devcontainer.json
generated_devcontainer = generate_dev_container_json(context)

Raw API Response:
ChatCompletion(id='chatcmpl-9g90SpSE7OtSPx7yalpZqidU1iDQf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Based on the provided context, here is a JSON configuration file (`devcontainer.json`) for a Visual Studio Code dev container to work with the Angular-Webpack project:\n\n```json\n{\n    "name": "Angular Webpack Dev Container",\n    "dockerFile": "Dockerfile",\n    "context": ".",\n    "settings": {\n        "terminal.integrated.shell.linux": "/bin/bash"\n    },\n    "extensions": [\n        "dbaeumer.vscode-eslint",\n        "esbenp.prettier-vscode",\n        "ms-vscode.vscode-typescript-tslint-plugin",\n        "angular.ng-template",\n        "eg2.tslint"\n    ],\n    "postCreateCommand": "npm install",\n    "runArgs": ["--init"],\n    "remoteUser": "vscode",\n    "overrideCommand": false,\n    "mounts": [\n        "source=/var/run/docker.sock,target=/var/run/docker.sock,type=bind"\n    ],\n    "containerEnv

In [93]:
# Main execution block
if flag == False:
    repos = generate_github_repos(prompt)
    files = retrieve_files(repos)
else:
    repo = [prompt]
    files = retrieve_files(repo)

Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.circleci/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.devcontainer/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/angular/angular/main/.github/actions/deploy-docs-site/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.vscode/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.yarn/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/adev/scripts/update-cli-help/README.md
Successfully retrieved README.md from https://raw.githubuserconte